In [1]:
import os
import re

def preprocess_text(text):
  # Replace multiple spaces with single spaces.
  text = re.sub(r'\s+', ' ', text)

  # Keep only letters, numbers, and spaces.
  text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

  return text

def chunk_text(text, chunk_size=512, overlap=30):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + chunk_size]
        chunks.append(' '.join(chunk))
        i += chunk_size - overlap  # step forward
    return chunks

# Load your legal document (txt file)
with open("/content/sample dataset.txt", "r", encoding="utf-8") as file:
    full_text = file.read()



# Chunk it
chunks = chunk_text(preprocess_text(full_text))

# Save the chunks to files (or keep in memory)
for idx, chunk in enumerate(chunks):
    with open(f"chunk_{idx}.txt", "w", encoding="utf-8") as f:
        f.write(chunk)

print(f"Created {len(chunks)} chunks.")

Created 51 chunks.


In [ ]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cud

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch



# Load pretrained T5 model
tokenizer = T5Tokenizer.from_pretrained("valhalla/t5-small-qg-hl")
model = T5ForConditionalGeneration.from_pretrained("valhalla/t5-small-qg-hl")

def generate_question(context):
    input_text = f"generate question: {context}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=64, num_beams=4, early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

# Example: generate one question per chunk
for i, chunk in enumerate(chunks[:2]):  # demo for first 5 chunks
    question = generate_question(chunk)
    print(f"\nChunk {i+1}:\nQ: {question}\nContext: {chunk[:]}...")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]


Chunk 1:
Q: What was the appeal of the High Court of Judicature at Bombay?
Context: Appeal No LXVI of 1949 Appeal from the High Court of judicature Bombay in a reference under section 66 of the Indian Income tax Act 1022 KM Munshi N P Nathvani with him for the appel lant MC Setalvad Attorney General for India H J Umrigar with him for the respondent 1950 May 26 The judgment of the Court was delivered by MEHR CHAND MAHAJAN J This is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 1 iv of the Indian Income tax Act The assessee company is an investment company deriving its income from properties in the city of Bombay For the assessment year 1940 41 the net income of the assessee under the head property was computed by the Income tax Officer in the sum of Rs 621764 after de

In [3]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# Load a question-answering pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Example: generate QA pair from chunk
qa_pairs = []

for i, chunk in enumerate(chunks):
    question = generate_question(chunk)
    result = qa_pipeline(question=question, context=chunk)
    answer = result['answer']

    qa_pairs.append({
        "question": question,
        "answer": answer,
        "context": chunk
    })

# Preview one
print(qa_pairs[0])


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'question': 'What was the appeal of the High Court of Judicature at Bombay?', 'answer': 'Appeal No LXVI of 1949', 'context': 'Appeal No LXVI of 1949 Appeal from the High Court of judicature Bombay in a reference under section 66 of the Indian Income tax Act 1022 KM Munshi N P Nathvani with him for the appel lant MC Setalvad Attorney General for India H J Umrigar with him for the respondent 1950 May 26 The judgment of the Court was delivered by MEHR CHAND MAHAJAN J This is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 1 iv of the Indian Income tax Act The assessee company is an investment company deriving its income from properties in the city of Bombay For the assessment year 1940 41 the net income of the assessee under the head property was computed by the Income ta

In [4]:
import pandas as pd

df = pd.DataFrame(qa_pairs)
df.to_csv("qa_pairs.csv", index=False)
print("question")
print(df['question'][1])
print("answer")
print(df['answer'][1])
print("context")
print(df['context'][1])


question
The Privy Council decided the case of Bijoy Singh Dudhuria vs Commissioner of Income tax Calcutta?
answer
assess ment on the general income
context
along with the relevant clause runs thus 1 The tax shall be payable by an assessee under the head income from property in respect of the bona fide annual value of property consisting of any buildings or lands appurtenant thereto of Which he is the owner subject to the following allowances namely iv where the property is subject to a mortgage or other capital charge the amount of any interest on such mortgage or charge where the property is subject to an annual charge not being a capital charge the amount of such charge where the property is subject to a ground rent the amount of such ground rent and where the property has been acquired constructed repaired renewed or recon structed with borrowed capital the amount of any interest payable on such capital It will be seen that clause iv consists of four sub clauses corresponding to th

In [5]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Initialize BPE tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Define special tokens
trainer = trainers.BpeTrainer(special_tokens=["<s>", "</s>", "<pad>", "<unk>", "<mask>"])

# Give the path to your single file
file_path = "/content/sample dataset.txt"  # Update with your actual file name

# Train tokenizer
tokenizer.train(files=[file_path], trainer=trainer)

# Save the tokenizer
tokenizer.save("legal_tokenizer.json")


In [ ]:
tokenizer = Tokenizer.from_file("legal_tokenizer.json")


print("Vocab size:", tokenizer.get_vocab_size())

sample_text = "one cast law"
encoding = tokenizer.encode(sample_text)

print("Tokens:", encoding.tokens)
print("IDs:", encoding.ids)





Vocab size: 4444
Tokens: ['one', 'cast', 'law']
IDs: [336, 2347, 629]


In [6]:
# Hyperparameters
VOCAB_SIZE = tokenizer.get_vocab_size()   # Update based on your tokenizer's vocab (2220 in your case)
MAX_LEN = 512       # Max input length
EMBED_DIM = 256     # Embedding size
NUM_HEADS = 8       # Number of attention heads
FF_DIM = 512        # Feedforward network size
NUM_LAYERS = 4      # Number of encoder and decoder layers
DROPOUT = 0.1

print(VOCAB_SIZE)


4444


In [7]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=512): # You might want to increase max_len here
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # Shape: (1, max_len, embed_dim)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, embed_dim)
        # If sequence length exceeds max_len, truncate positional encodings
        seq_len = x.size(1)
        #Adjusting the max_len parameter or truncating/padding sequences to ensure compatibility.
        #Increasing max_len to accommodate the longer sequence.
        pe = self.pe[:, :min(seq_len, self.pe.size(1))] # Truncate if needed
        if seq_len > self.pe.size(1):
            # If the sequence length is greater than max_len, repeat the positional encodings
            # to cover the entire sequence length
            num_repeats = seq_len // self.pe.size(1) + 1
            pe = self.pe.repeat(1, num_repeats, 1)[:, :seq_len, :]

        return x + pe # Add the positional encodings


In [8]:
import torch.nn.functional as F

class EncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        # Self-attention
        attn_output, _ = self.self_attn(x, x, x, key_padding_mask=src_mask)
        x = self.norm1(x + self.dropout(attn_output))  # Residual + Norm

        # Feedforward
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout(ff_output))  # Residual + Norm
        return x


In [9]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, tgt_mask=None, memory_mask=None):
        # 1. Masked Self-Attention (for decoder input)
        attn1, _ = self.self_attn(x, x, x, attn_mask=tgt_mask)
        x = self.norm1(x + self.dropout(attn1))

        # 2. Cross-Attention (decoder attends to encoder output)
        attn2, _ = self.cross_attn(x, enc_output, enc_output, attn_mask=memory_mask)
        x = self.norm2(x + self.dropout(attn2))

        # 3. Feedforward
        ff_out = self.ff(x)
        x = self.norm3(x + self.dropout(ff_out))
        return x


In [10]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, num_heads=8, ff_dim=2048, num_layers=6, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim

        # Embeddings
        self.src_embedding = nn.Embedding(vocab_size, embed_dim)
        self.tgt_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim)

        # Encoder
        self.encoder_layers = nn.ModuleList([
            EncoderBlock(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # Decoder
        self.decoder_layers = nn.ModuleList([
            DecoderBlock(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # Final output layer
        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, input_ids, decoder_input_ids, src_mask=None, tgt_mask=None, memory_mask=None): # Modified to take input_ids and decoder_input_ids
        # Embed and add position
        # Assume input_ids are the source sequence
        src = self.pos_encoder(self.src_embedding(input_ids))
        # Assume decoder_input_ids are the target sequence
        tgt = self.pos_encoder(self.tgt_embedding(decoder_input_ids))

        # Encoder
        for layer in self.encoder_layers:
            src = layer(src, src_mask)

        # Decoder
        output = tgt
        for layer in self.decoder_layers:
            output = layer(output, src, tgt_mask, memory_mask)

        return self.output_layer(output) # Changed to return output logits directly

In [11]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class LegalQADatasetCSV(Dataset):
    def __init__(self, path, tokenizer, max_len=512):
        self.data = pd.read_csv(path)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.loc[idx, 'question']
        answer = self.data.loc[idx, 'answer']
        context = self.data.loc[idx, 'context']

        src_text = question + " [SEP] " + context
        tgt_text = answer

        src_tokens = self.tokenizer.encode(src_text).ids[:self.max_len]
        tgt_tokens = self.tokenizer.encode(tgt_text).ids[:self.max_len]

        decoder_input = [self.tokenizer.token_to_id("<s>")] + tgt_tokens[:-1]
        decoder_target = tgt_tokens

        return {
            "src": torch.tensor(src_tokens, dtype=torch.long),
            "tgt_in": torch.tensor(decoder_input, dtype=torch.long),
            "tgt_out": torch.tensor(decoder_target, dtype=torch.long)
        }


In [12]:


from torch.nn.utils.rnn import pad_sequence # Import pad_sequence

def collate_fn(batch):
    src_batch = [item["src"] for item in batch]
    tgt_in_batch = [item["tgt_in"] for item in batch]
    tgt_out_batch = [item["tgt_out"] for item in batch]

    src_pad = pad_sequence(src_batch, batch_first=True, padding_value=tokenizer.token_to_id("<pad>"))
    tgt_in_pad = pad_sequence(tgt_in_batch, batch_first=True, padding_value=tokenizer.token_to_id("<pad>"))
    tgt_out_pad = pad_sequence(tgt_out_batch, batch_first=True, padding_value=tokenizer.token_to_id("<pad>"))

    # Return the padded tensors, not the keys
    return src_pad, tgt_in_pad, tgt_out_pad


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

# ⚙️ Hyperparameters
batch_size = 8
epochs = 10
learning_rate = 3e-4
#Instead of -1, set to 'cpu' if cuda is not available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pad_id = tokenizer.token_to_id("<pad>")


# 🧱 Create dataset and dataloader
dataset = LegalQADatasetCSV("/content/qa_pairs.csv", tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)



# 🧠 Model, loss, optimizer
model = TransformerModel(vocab_size=tokenizer.get_vocab_size(), embed_dim=512, num_heads=8, num_layers=6).to(device)

# Changed 'd_model' to 'embed_dim'

# criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
criterion= nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id("<pad>"),label_smoothing=0.1)

optimizer = AdamW(model.parameters(), lr=learning_rate)

In [14]:
import torch

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# Inside your training loop:
for epoch in range(10):
    # ... (other code)

    for batch in dataloader:
        # unpack the batch here (src, tgt_in, tgt_out)
        src, tgt_in, tgt_out = batch
        src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)


        # 📌 Generate attention masks (Corrected)
        src_mask = (src == pad_id)  # shape: (B, S)
        tgt_mask = (tgt_in == pad_id)  # shape: (B, T)

        # Generate square causal mask for decoder self-attention
        tgt_seq_len = tgt_in.shape[1]
        tgt_attn_mask = generate_square_subsequent_mask(tgt_seq_len).to(device)

        # 🔄 Forward pass
        output = model(src, tgt_in, src_mask=src_mask, tgt_mask=tgt_attn_mask)  # Pass tgt_attn_mask here
        # ... (rest of the training loop)

In [16]:
for epoch in range(50):
    model.train()
    total_loss = 0

    for batch in dataloader:
        src, tgt_in, tgt_out = batch
        src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)

        # 📌 Generate attention masks
        src_mask = (src == pad_id)  # shape: (B, S)
        tgt_mask = generate_square_subsequent_mask(tgt_in.shape[1]).type(torch.bool).to(device) # shape: (T, T)

        # 🔄 Forward pass
        output = model(src, tgt_in, src_mask=src_mask, tgt_mask=tgt_mask)

        # 🎯 Flatten for loss computation
        output = output.view(-1, output.size(-1))
        tgt_out = tgt_out.view(-1)

        loss = criterion(output, tgt_out)

        # 🔁 Backward + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)

    print(f"Epoch [{epoch+1}/{100}] - Loss: {avg_loss:.4f}")



Epoch [1/100] - Loss: 5.4795
Epoch [2/100] - Loss: 5.5062
Epoch [3/100] - Loss: 5.5561
Epoch [4/100] - Loss: 5.5131
Epoch [5/100] - Loss: 5.4797
Epoch [6/100] - Loss: 5.5107
Epoch [7/100] - Loss: 5.5025
Epoch [8/100] - Loss: 5.5168
Epoch [9/100] - Loss: 5.4756
Epoch [10/100] - Loss: 5.5124
Epoch [11/100] - Loss: 5.5264
Epoch [12/100] - Loss: 5.5158
Epoch [13/100] - Loss: 5.4439
Epoch [14/100] - Loss: 5.4520
Epoch [15/100] - Loss: 5.3286
Epoch [16/100] - Loss: 5.0030
Epoch [17/100] - Loss: 4.4230
Epoch [18/100] - Loss: 3.7414
Epoch [19/100] - Loss: 3.1035
Epoch [20/100] - Loss: 2.6918
Epoch [21/100] - Loss: 2.3271
Epoch [22/100] - Loss: 2.0947
Epoch [23/100] - Loss: 2.1882
Epoch [24/100] - Loss: 1.9638
Epoch [25/100] - Loss: 1.8727
Epoch [26/100] - Loss: 1.8181
Epoch [27/100] - Loss: 1.9618
Epoch [28/100] - Loss: 1.8224
Epoch [29/100] - Loss: 1.8644
Epoch [30/100] - Loss: 1.8273
Epoch [31/100] - Loss: 1.8224
Epoch [32/100] - Loss: 1.7964
Epoch [33/100] - Loss: 1.8073
Epoch [34/100] - Lo

In [19]:
torch.save(model.state_dict(), "transformer_legalQA1.pt")


In [20]:
def generate_answer(model, tokenizer, question, context, max_len=64):
    model.eval()
    with torch.no_grad():
        src_text = question + " [SEP] " + context
        src_ids = torch.tensor([tokenizer.encode(src_text).ids]).to(device)

        generated = [tokenizer.token_to_id("<s>")]

        for _ in range(max_len):
            tgt_tensor = torch.tensor([generated]).to(device)
            output = model(input_ids=src_ids, decoder_input_ids=tgt_tensor)
            next_token_logits = output[:, -1, :]

            # Greedy decoding: Select the token with the highest probability
            next_token = torch.argmax(next_token_logits, dim=-1).item()

            generated.append(next_token)

            if next_token == tokenizer.token_to_id("</s>"):
                break

        decoded = tokenizer.decode(generated[1:])  # skip <s>
        return decoded

In [21]:
!pip install -q faiss-cpu sentence-transformers
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


# 1. Create a SentenceTransformer for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Embed all your document chunks and create a FAISS index
document_chunks = chunks  # Assuming all_contexts holds your chunked data
document_embeddings = embedder.encode(document_chunks, convert_to_tensor=True)
document_embeddings = document_embeddings.cpu().numpy()  # Convert to NumPy for FAISS

dimension = document_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings)



def retrieve_and_generate(query, model, tokenizer, k=5):
    # 🔍 Step 1: Embed the query using a SentenceTransformer
    query_vec = embedder.encode(query)

    # 🧠 Step 2: Search FAISS index
    _, top_k_idx = faiss_index.search(np.array([query_vec]), k=k)

    # 🧾 Step 3: Retrieve top-k chunks
    retrieved_chunks = [document_chunks[i] for i in top_k_idx[0]]

    # 📃 Step 4: Concatenate retrieved chunks
    combined_context = " ".join(retrieved_chunks)

    # 🧠 Step 5: Use query + retrieved context as input to your Transformer
    # Call generate_answer, which uses model.forward()
    return generate_answer(model, tokenizer, query, combined_context, max_len=100)

# ... (rest of your code, including the call to retrieve_and_generate)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
k=input()
print(retrieve_and_generate(k, model, tokenizer, k=3))


The Privy Council decided the case of Bijoy Singh Dudhuria vs Commissioner of Income tax Calcutta?
the misconduct of the respondent of the respondent misconduct of the respondent of the respondent of the respondent of the respondent lands had been made certain amendments in the respondent of the respondent of the respondent of the respondent of the respondent to appropriate to himself to himself of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent of the respondent Government
